In [ ]:
import sqlite3
db = sqlite3.connect("APAD_proj.db")

In [ ]:
def new_user(db):


    cursor = db.cursor()
    

    while(1):
        email = str(input("Please provide your email-address: "))
        findEmail = '''SELECT * FROM user where email_id = ?'''

        cursor.execute(findEmail, (email,))

        if cursor.fetchall():
            print("Email already registered, enter again")

        else:

            first_name = str(input("Please Enter your first name: "))
            last_name = str(input("Please Enter your last name: "))
            contact_number = str(input("Please provide a valid contact number: "))
            zip_code = str(input("Your zip code: "))
            password = str(input("Enter the password: ")) # NEED to add constraints
            password_check = str(input("Enter the password again to recheck: "))

            while password!=password_check:
                password = str(input("Your passwords did not match, try again: "))
                password_check = str(input("Enter the password again to recheck: "))

            query = '''INSERT INTO user(email_id,first_name,last_name,cotact_number, zip_code, password)
                            VALUES(?,?,?,?,?,?)'''
                
            cursor.execute(query, (email, first_name, last_name, contact_number, zip_code, password))  
            db.commit()
            break


In [ ]:
new_user(db)

In [ ]:
def new_venue(db, admin):
    
    cursor = db.cursor()

    email = str(input("Please enter your email address: "))
    
    while email != admin:
        print("You are not authorized to add a venue")
        email = str(input("Enter your email address again: "))
    
    venue_name = str(input("Please Enter the venue name: "))
    address = str(input("Please Enter the venue address: "))
    zip_code = str(input("Venue zip code: "))
    contact_number = str(input("Please provide a contact number: "))
    description = str(input("Enter the venue description (not more than 200 characters): ")) # Need to limit the length of input

    query = '''INSERT INTO venue(venue_name, address, zip_code, contact_number, description)
                VALUES(?,?,?,?,?)'''

    # cursor = db.cursor()
    cursor.execute(query, (venue_name, address, zip_code, contact_number, description)) 
    db.commit()


In [ ]:
admin_id = "prajvalgupta@gmail.com"

new_venue(db, admin_id)


In [29]:
import sqlite3
db = sqlite3.connect("APAD_proj.db")

In [30]:
def new_event(db):
    cursor = db.cursor()
    
    email = str(input("Please provide your email-address: "))
    findEmail = '''SELECT email_id FROM user where email_id = ?'''
    cursor.execute(findEmail, (email,))
    create_or_join = str(input("Do you want to create/join the game: ")).lower()
    
    if cursor.fetchall():
        if create_or_join == "create":
            event_name = str(input("Please Enter the event name: "))
            event_date = str(input("Please Enter the event date: "))
            start_time = str(input("Please Enter the start time in hrs: "))
            end_time   = str(input("Please Enter the end time in hrs: "))
            event_desc = str(input("Please Enter the event description: "))
            capacity   = str(input("Please Enter the capacity avail: "))
            event_category = str(input("Please Enter the event category as watching/playing/mentoring: "))
            
            host_insert_flag = '''INSERT INTO events(event_cat_id,venue_id,event_name, date, start_time,end_time,
                                                    user_id, host_flag, member_flag, sports_cat_id, event_desc, capacity_avail)
                            VALUES(?,?,?,?,?,?,?,?,?,?,?,?)'''
                
            #cursor.execute(host_insert_flag, (...,...,event_name,event_date,start_time,end_time,...,1,0, ...., event_desc, capacity))  
            #need to write a sub query for missing columns and fill up event_cat_id table, 
            #need to update main fact_table. with email id then join to all other
            #db.commit()
            
            # need to do host_flag as 1
        elif create_or_join == "join":
            event_date = str(input("Please Enter the event date: "))
            start_time = str(input("Please Enter the start time in hrs: "))
            end_time   = str(input("Please Enter the end time in hrs: "))
            find_event = '''SELECT * FROM events where date = ?
                                and start_time = ? and end_time = ?'''
            cursor.execute(find_event, (event_date, start_time, end_time,))
            p = cursor.fetchall()
        else:
            print("enter a valid option")
            
    else:
        print("you can't create an event yet. Create an account first")
    

In [31]:
new_event(db)

Please provide your email-address: ritika.munjal29@gmail.com
Do you want to create/join the game: join
Please Enter the event date: 12
Please Enter the start time in hrs: 1
Please Enter the end time in hrs: 12
[]
